In [1]:
import numpy as np
import pandas as pd
import os
import joblib
from functools import partial
from tqdm import notebook, tqdm
from scipy.sparse import coo_matrix
# from coo import co_acc_probs, co_acc_skills
# multiprocessing to speedup matrix extraction
import multiprocessing

# cores = multiprocessing.cpu_count()
cores = 20

pool = multiprocessing.Pool(processes=cores)

In [2]:
# read in the data
df = pd.read_csv('./assistment2017.csv', low_memory=False)
df

,studentId,MiddleSchoolId,InferredGender,SY ASSISTments Usage,AveKnow,AveCarelessness,AveCorrect,NumActions,AveResBored,AveResEngcon,...,RES_CONFUSED,RES_FRUSTRATED,RES_OFFTASK,RES_GAMING,Ln-1,Ln,MCAS,Enrolled,Selective,isSTEM
0,8,2,Male,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,...,0.000000,0.000000,0.785585,0.000264,0.13,0.061190409,45,0,0,NaN
1,8,2,Male,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,...,0.887452,0.000000,0.468252,0.001483,0.061190409,0.213509945,45,0,0,NaN
2,8,2,Male,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,...,0.887452,0.000000,0.468252,0.001483,0.116,0.033305768,45,0,0,NaN
3,8,2,Male,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,...,0.000000,0.000000,0.108417,0.010665,0.116,0.033305768,45,0,0,NaN
4,8,2,Male,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,...,0.000000,0.000000,0.108417,0.010665,0.033305768,0.118385889,45,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942811,7783,1,NaN,2005-2006,0.210902,0.094888,0.456522,276,0.271812,0.611678,...,0.000000,0.000000,0.122595,0.000092,0.054517257,0.053783108,28,1,0,NaN
942812,7783,1,NaN,2005-2006,0.210902,0.094888,0.456522,276,0.271812,0.611678,...,0.633474,0.000000,0.122595,0.002815,0.053783108,0.053673232,28,1,0,NaN
942813,7783,1,NaN,2005-2006,0.210902,0.094888,0.456522,276,0.271812,0.611678,...,0.000000,0.000000,0.122595,0.101184,0.053783108,0.053673232,28,1,0,NaN
942814,7783,1,NaN,2005-2006,0.210902,0.094888,0.456522,276,0.271812,0.611678,...,0.000000,0.000000,0.122595,0.101184,0.053783108,0.053673232,28,1,0,NaN


In [19]:
df.columns

Index(['studentId', 'MiddleSchoolId', 'InferredGender', 'SY ASSISTments Usage',
       'AveKnow', 'AveCarelessness', 'AveCorrect', 'NumActions', 'AveResBored',
       'AveResEngcon', 'AveResConf', 'AveResFrust', 'AveResOfftask',
       'AveResGaming', 'action_num', 'skill', 'problemId', 'problemType',
       'assignmentId', 'assistmentId', 'startTime', 'endTime', 'timeTaken',
       'correct', 'original', 'hint', 'hintCount', 'hintTotal', 'scaffold',
       'bottomHint', 'attemptCount', 'frIsHelpRequest', 'frPast5HelpRequest',
       'frPast8HelpRequest', 'stlHintUsed', 'past8BottomOut',
       'totalFrPercentPastWrong', 'totalFrPastWrongCount', 'frPast5WrongCount',
       'frPast8WrongCount', 'totalFrTimeOnSkill', 'timeSinceSkill',
       'frWorkingInSchool', 'totalFrAttempted', 'totalFrSkillOpportunities',
       'responseIsFillIn', 'responseIsChosen', 'endsWithScaffolding',
       'endsWithAutoScaffolding', 'frTimeTakenOnScaffolding',
       'frTotalSkillOpportunitiesScaffolding',
 

In [6]:
df[df['problemType'] != "noprotype"]['problemType'].tolist()

['textfieldquestion',
 'noprobtype',
 'noprobtype',
 'noprobtype',
 'noprobtype',
 'noprobtype',
 'noprobtype',
 'radioquestion',
 'noprobtype',
 'noprobtype',
 'noprobtype',
 'noprobtype',
 'noprobtype',
 'noprobtype',
 'noprobtype',
 'noprobtype',
 'radioquestion',
 'textfieldquestion',
 'textfieldquestion',
 'textfieldquestion',
 'textfieldquestion',
 'textfieldquestion',
 'textfieldquestion',
 'radioquestion',
 'textfieldquestion',
 'radioquestion',
 'radioquestion',
 'radioquestion',
 'textfieldquestion',
 'radioquestion',
 'radioquestion',
 'radioquestion',
 'textfieldquestion',
 'textfieldquestion',
 'textfieldquestion',
 'textfieldquestion',
 'textfieldquestion',
 'radioquestion',
 'radioquestion',
 'other',
 'other',
 'other',
 'other',
 'noprobtype',
 'noprobtype',
 'noprobtype',
 'noprobtype',
 'other',
 'noprobtype',
 'noprobtype',
 'noprobtype',
 'textfieldquestion',
 'noprobtype',
 'noprobtype',
 'noprobtype',
 'radioquestion',
 'noprobtype',
 'noprobtype',
 'noprobtype',

In [3]:
 # preprocess assist2017
read_col = ['studentId', 'assignmentId', 'assistmentId', 'problemId', 'correct','skill', 'original']
target = 'correct'

df = df[read_col]
df['skill_id'], _ = pd.factorize(df['skill'])
# df

/tmp/ipykernel_19370/452367036.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['skill_id'], _ = pd.factorize(df['skill'])


In [4]:
# delete empty skill_id
df = df.dropna(subset=['skill_id'])
df = df[~df['skill_id'].isin(['noskill'])]
df.skill_id = df.skill_id.astype('int')
print('After removing empty skill_id, records number %d' % len(df))

After removing empty skill_id, records number 942816


In [22]:
df[df['original'].isin([1])]

,studentId,assignmentId,assistmentId,problemId,correct,skill,original,skill_id
0,8,20405010,104051118,1118,0,properties-of-geometric-figures,1,72
7,8,20405010,104051088,1088,0,point-plotting,1,69
16,8,20405010,104051074,1074,1,reading-graph,1,78
17,8,20405010,104051204,1204,0,area,1,14
22,8,20405010,104051182,1182,0,area,1,14
...,...,...,...,...,...,...,...,...
942782,7783,723,441,441,1,symbolization-articulation,1,96
942783,7783,723,4003,4003,0,exponents,1,30
942787,7783,723,1644,1644,0,proportion,1,74
942793,7783,723,2274,2274,0,mean,1,50


In [5]:
# delete scaffolding problems
df = df[df['original'].isin([1])]
print('After removing scaffolding problems, records number %d' % len(df))

After removing scaffolding problems, records number 249105


In [6]:
df.rename(columns={'studentId':'user_id', 'problemId':'problem_id'}, inplace=True)
# df

In [7]:
min_inter_num = 3
users = df.groupby(['user_id'], as_index=True)
delete_users = []
for u in users:
    if len(u[1]) < min_inter_num:
        delete_users.append(u[0])
print('deleted user number based min-inters %d' % len(delete_users))
df = df[~df['user_id'].isin(delete_users)]
df = df[[ 'user_id', 'problem_id', 'skill_id', 'correct']]
print('After deleting some users, records number %d' % len(df))

deleted user number based min-inters 3
After deleting some users, records number 249102


In [26]:
df

,user_id,problem_id,skill_id,correct
0,8,1118,72,0
7,8,1088,69,0
16,8,1074,78,1
17,8,1204,14,0
22,8,1182,14,0
...,...,...,...,...
942782,7783,441,96,1
942783,7783,4003,30,0
942787,7783,1644,74,0
942793,7783,2274,50,0


In [8]:
# 5,"[55969, 55970, ...
skill_df = df[['skill_id', 'problem_id']].groupby(['skill_id'], as_index=True).apply(lambda r: np.array(list(set(r['problem_id'].values))))
# print(skill_df)
joblib.dump(skill_df, './a17/skill_prob.pkl.zip')
user_prob = df[['user_id', 'problem_id', 'correct']].groupby(['user_id', 'problem_id'])['correct'].agg('mean')


In [9]:
def co_acc_probs(user_prob, prob1, prob2):
    count = 0
    agg = 0
    for user, prob in user_prob.index:
        if prob == prob1:
            if prob2 in user_prob[user].index:
                count += 1
                agg += user_prob[user][prob1] * user_prob[user][prob2]
#                 print('user {} answered two questions, with {} answered {}, and {} answered {}'
#                   .format(user, prob1, user_prob[user][prob1], prob2, user_prob[user][prob2]))
    if count == 0:
        return -1
    else:
        return agg/count


def co_acc_skills(user_skill, skill1, skill2):
    count = 0
    agg = 0
    for user, skill in user_skill.index:
        if skill1 == skill: 
            if skill2 in user_skill[user].index:
                count += 1
                agg += user_skill[user][skill1] * user_skill[user][skill2]
    if count == 0:
        return -1
    else:
        return agg/count
        

In [10]:
# e2e matrix
skill_mats = []
e2e = partial(co_acc_probs, user_prob)
# e2e

In [ ]:
skill_df.index

## Extract concept-to-concept matrix and exercise -to-exercise matrix

In [11]:
pool = multiprocessing.Pool(processes=20)
for skill in tqdm(skill_df.index):
    print('processing skill: ', skill)
    skill_probs = skill_df[skill]
    row = []
    col = []
    args = ((prob1, prob2) for i, prob1 in enumerate(skill_probs) for prob2 in skill_probs[i:])
    val = pool.starmap(e2e, args)
    print('matrix size is: ', len(val))
    for i in range(len(skill_probs)):
        for j in range(i, len(skill_probs)):
            row.append(i)
            col.append(j)
    assert(len(val)==len(row))
    mat = coo_matrix((val, (row, col)), shape=(len(skill_probs), len(skill_probs)))
    skill_mats.append(mat)
joblib.dump(skill_mats, './a17/nodel/para_e2e.pkl.zip')

  0%|          | 0/86 [00:00<?, ?it/s]

processing skill:  0


  1%|          | 1/86 [00:08<11:23,  8.04s/it]

matrix size is:  55
processing skill:  1


  2%|▏         | 2/86 [00:13<09:15,  6.61s/it]

...

 99%|█████████▉| 85/86 [33:27<00:01,  1.20s/it]

matrix size is:  1
processing skill:  101


100%|██████████| 86/86 [33:28<00:00, 23.35s/it]

matrix size is:  1


['./a17/nodel/para_e2e.pkl.zip']

In [12]:
skill_acc = df[['skill_id', 'correct']].groupby('skill_id')['correct'].agg('mean')
user_skill = df[['user_id', 'skill_id', 'correct']].groupby(['user_id', 'skill_id'])['correct'].agg('mean')

for user, skill in tqdm(user_skill.index):
    if user_skill[user][skill]>=skill_acc[skill]:
        user_skill[user][skill] = 1
    else:
        user_skill[user][skill] = 0

100%|██████████| 60175/60175 [01:38<00:00, 608.77it/s] 


In [13]:
pool = multiprocessing.Pool(processes=20)
        
# c2c matrix
row = []
col = []
val = []
print('Extracting c2c matrix...')
arg_skills = ((skill1, skill2) for (i, skill1) in enumerate(skill_df.index) for skill2 in skill_df.index[i:])
c2c = partial(co_acc_skills, user_skill)
val = pool.starmap(c2c, arg_skills)
for i in range(len(skill_df.index)):
    for j in range(i, len(skill_df.index)):
        row.append(i)
        col.append(j)
mat_skill = coo_matrix((val, (row, col)), shape=(len(skill_df.index), len(skill_df.index)))
joblib.dump(mat_skill,'./a17/nodel/c2c.pkl.zip')

Extracting c2c matrix...


['./a17/nodel/c2c.pkl.zip']

In [26]:
df = df[~df['user_id'].isin(delete_users)]
df = df[['user_id', 'problem_id', 'skill_id', 'timestep', 'correct']]
print('After deleting some users, records number %d' % len(df))

After deleting some users, records number 219716


In [27]:
skill_df = df[['skill_id', 'problem_id']].groupby(['skill_id'], as_index=True).apply(
        lambda r: np.array(list(set(r['problem_id'].values))))
joblib.dump(skill_df, './data/a17/skill_prob.pkl.zip')

['./data/a17/skill_prob.pkl.zip']

In [28]:
df[['user_id', 'problem_id', 'skill_id', 'timestep','correct']]

,user_id,problem_id,skill_id,timestep,correct
0,8,1118,71,2004-09-29 15:05:01,0
7,8,1088,68,2004-09-29 15:07:34,0
16,8,1074,77,2004-09-29 15:11:07,1
17,8,1204,14,2004-09-29 15:12:30,0
22,8,1182,14,2004-09-29 15:14:14,0
...,...,...,...,...,...
942782,7783,441,95,2006-05-12 15:14:38,1
942783,7783,4003,30,2006-05-12 15:15:53,0
942787,7783,1644,73,2006-05-12 15:17:06,0
942793,7783,2274,50,2006-05-12 15:18:48,0


In [39]:
def co_acc_probs(user_prob, prob1, prob2):
    '''共同出现的问题'''
    count = 0
    agg = 0
    for user, prob in user_prob.index:
        if prob == prob1:
            if prob2 in user_prob[user].index:
                count += 1
                agg += user_prob[user][prob1] * user_prob[user][prob2]
#                 print('user {} answered two questions, with {} answered {}, and {} answered {}'
#                   .format(user, prob1, user_prob[user][prob1], prob2, user_prob[user][prob2]))
    if count == 0:
        return -1
    else:
        return agg/count

def co_acc_skills(user_skill, skill1, skill2):
    '''共同出现的知识点'''
    count = 0
    agg = 0
    for user, skill in user_skill.index:
        if skill1 == skill:
            if skill2 in user_skill[user].index:
                count += 1
                agg += user_skill[user][skill1] * user_skill[user][skill2]
    if count == 0:
        return -1
    else:
        return agg/count

In [ ]:
user_prob = df[['user_id', 'problem_id', 'correct']].groupby(['user_id', 'problem_id'])['correct'].agg('mean')

# e2e matrix
skill_mats = []
e2e = partial(co_acc_probs, user_prob)
for skill in tqdm(skill_df.index):
    print('processing skill: ', skill)
    skill_probs = skill_df[skill]
    row = []
    col = []
    args = ((prob1, prob2) for i, prob1 in enumerate(skill_probs) for prob2 in skill_probs[i:])
    val = pool.starmap(e2e, args)
    print('matrix size is: ', len(val))
    for i in range(len(skill_probs)):
        for j in range(i, len(skill_probs)):
            row.append(i)
            col.append(j)
    assert (len(val) == len(row))
    mat = coo_matrix((val, (row, col)), shape=(len(skill_probs), len(skill_probs)))
    skill_mats.append(mat)
joblib.dump(skill_mats, './data/a17/para_e2e.pkl.zip')

In [41]:
skill_acc = df[['skill_id', 'correct']].groupby('skill_id')['correct'].agg('mean')
user_skill = df[['user_id', 'skill_id', 'correct']].groupby(['user_id', 'skill_id'])['correct'].agg('mean')

for user, skill in tqdm(user_skill.index):
    if user_skill[user][skill] >= skill_acc[skill]:
        user_skill[user][skill] = 1
    else:
        user_skill[user][skill] = 0


100%|██████████████████████████████████████████████████████████████████████| 59144/59144 [00:28<00:00, 2039.86it/s]


In [42]:
# c2c matrix
row = []
col = []
val = []
print('Extracting c2c matrix...')
arg_skills = ((skill1, skill2) for (i, skill1) in enumerate(skill_df.index) for skill2 in skill_df.index[i:])
c2c = partial(co_acc_skills, user_skill)
val = pool.starmap(c2c, arg_skills)
for i in range(len(skill_df.index)):
    for j in range(i, len(skill_df.index)):
        row.append(i)
        col.append(j)
mat_skill = coo_matrix((val, (row, col)), shape=(len(skill_df.index), len(skill_df.index)))
joblib.dump(mat_skill, './data/a17/c2c.pkl.zip')


Extracting c2c matrix...


['./data/a17/c2c.pkl.zip']

In [20]:
skill_df[skill_df.index == 72].tolist()

[array([ 896, 2239, 1059, 2244,  935, 2289,  850, 2201, 2267,  893,  575])]

In [6]:
import joblib
skill_prob = joblib.load('./a17/skill_prob.pkl.zip')
skill_prob

filtered_skill_prob = {}
channel = 10
for i, skill_id in enumerate(skill_prob.index):
    if len(skill_prob[skill_id]) >= channel:
        filtered_skill_prob[skill_id] = skill_prob[skill_id]

In [ ]:
import numpy as np

data_path = './a17/emb'
e2e_emb = joblib.load(f'{data_path}/e2e_emb.pkl.zip')
c2c_emb = joblib.load(f'{data_path}/c2c_emb.pkl.zip')

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
all_c_v = []
for k, v in c2c_emb.items():
    all_c_v.extend(list(v.numpy()))
all_c_v = scaler.fit_transform(np.array(all_c_v).reshape(-1, 1))

all_c_v1 = {}
for i, (k, v) in enumerate(c2c_emb.items()):
    all_c_v1[k] = all_c_v[i*10:(i+1)*10].reshape(-1,)
    
c2c_emb

In [ ]:
all_e_v = {}
for skill, qu_embs in e2e_emb.items():
    q_num = qu_embs.shape[0]
    temp_all_v = qu_embs.numpy().reshape(-1,)
    temp_all_v = scaler.fit_transform(np.array(temp_all_v).reshape(-1, 1))
    all_e_v[skill] = temp_all_v.reshape(-1, 10)
all_e_v

In [15]:
skill_emb = {}
for skill in filtered_skill_prob.keys():
    print(skill)
    temp_c = (np.array(all_c_v1[skill]))
    temp_e = np.array(np.mean(all_e_v[skill], axis=0))
    skill_emb[skill] = np.append(temp_c, temp_e)

1
3


KeyError: 3